<center><img src="images/DLI Header.png" alt="Header" width="400"></center>

# 5.0 Using Different Neural Networks (Optional)
### INTERNET CONNECTION TO JETSON NANO REQUIRED

In the examples presented so far, the object detector was a ResNet10 network trained to detect four different objects (Vehicle, Bicycle, Person, Roadsign). In many cases, you may wish to use a different network for other tasks. DeepStream natively supports other networks including YOLO, SSD, and Faster-RCNN. For experienced developers, additional networks can be integrated with DeepStream through TensorRT.  

To use these networks, there is some setup that requires internet access.  The easiest way to accomplish this is via the Ethernet port on the Jetson Nano. In this notebook, you'll learn how to incorporate a YOLO network into a DeepStream app.

<img src="images/05_example.png" alt="YOLO network pipeline">

5.1 **[YOLO](#05_yolo)**<br>
&nbsp; &nbsp; &nbsp; 5.1.1 [Set Up YOLO with DeepStream SDK](#05_yolos_setup)<br>
&nbsp; &nbsp; &nbsp; 5.1.2 [Exercise: Build and Run a YOLO Application](#05_ex_yolo)<br>
&nbsp; &nbsp; &nbsp; 5.1.3 [Exercise: Annotate Your Own Video with YOLO](#05_ex_your_video)<br>
5.2 **[Faster-RCNN](#05_faster_rcnn)**<br>
5.3 **[SSD](#05_ssd)**<br>

<a name="05_yolo"></a>
# 5.1 YOLO
The YOLO, or "You Only Look Once", network employs a single convolutional network that simultaneously predicts multiple bounding boxes and class probabilities for those boxes.  DeepStream SDK includes support for YOLO in the `/home/dlinano/deepstream_sdk_v4.0.2_jetson/sources/objectDetector_Yolo` directory. Navigate there now using the JupyterLab file browser. Click the `labels.txt` file to view the ordered list of 80 objects that can be detected by this network!

Before using it the first time, you'll need to run the setup scripts to download weight and configuration files.

<a name="05_setup_yolo"></a>
## 5.1.1 Set Up YOLO with DeepStream SDK
Follow these steps to set up YOLO on your Jetson Nano.  Your Jetson Nano must be connected to the Internet.
1. Navigate to `/home/dlinano/deepstream_sdk_v4.0.2_jetson/sources/objectDetector_Yolo/nvdsinfer_custom_impl_Yolo` with the JupyterLab file browser.
2. Click on `Makefile` to edit.
   * Set the CUDA version:  `CUDA_VER=10.0`
   * Save and close the file.
3. Execute the following cells to make and prebuild YOLO.

In [1]:
%cd /home/dlinano/deepstream_sdk_v4.0.2_jetson/sources/objectDetector_Yolo/nvdsinfer_custom_impl_Yolo/
!make

/home/dlinano/deepstream_sdk_v4.0.2_jetson/sources/objectDetector_Yolo/nvdsinfer_custom_impl_Yolo
g++ -c -o nvdsinfer_yolo_engine.o -Wall -std=c++11 -shared -fPIC -Wno-error=deprecated-declarations -I../../includes -I/usr/local/cuda-10.0/include nvdsinfer_yolo_engine.cpp
g++ -c -o nvdsparsebbox_Yolo.o -Wall -std=c++11 -shared -fPIC -Wno-error=deprecated-declarations -I../../includes -I/usr/local/cuda-10.0/include nvdsparsebbox_Yolo.cpp
g++ -c -o yoloPlugins.o -Wall -std=c++11 -shared -fPIC -Wno-error=deprecated-declarations -I../../includes -I/usr/local/cuda-10.0/include yoloPlugins.cpp
g++ -c -o trt_utils.o -Wall -std=c++11 -shared -fPIC -Wno-error=deprecated-declarations -I../../includes -I/usr/local/cuda-10.0/include trt_utils.cpp
trt_utils.cpp: In function ‘void displayDimType(nvinfer1::Dims)’:
trt_utils.cpp:126:19: warning: ‘nvinfer1::Dims::type’ is deprecated [-Wdeprecated-declarations]
         switch (d.type[i])
                   ^~~~
In file included from /usr/include/aarch64

In [2]:
%cd /home/dlinano/deepstream_sdk_v4.0.2_jetson/sources/objectDetector_Yolo/
!./prebuild.sh

/home/dlinano/deepstream_sdk_v4.0.2_jetson/sources/objectDetector_Yolo
yolov2.cfg          100%[===================>]   2.66K  --.-KB/s    in 0.001s  
yolov2.weights      100%[===================>] 194.49M   114KB/s    in 27m 17s 
yolov2-tiny.cfg     100%[===================>]   1.45K  --.-KB/s    in 0.001s  
yolov2-tiny.weights 100%[===================>]  42.87M   119KB/s    in 6m 27s  
yolov3.cfg          100%[===================>]   8.15K  --.-KB/s    in 0s      
yolov3.weights      100%[===================>] 236.52M   834KB/s    in 26m 37s 
yolov3-tiny.cfg     100%[===================>]   1.87K  --.-KB/s    in 0.001s  
yolov3-tiny.weights 100%[===================>]  33.79M   647KB/s    in 1m 40s  


<a name="05_ex_yolo"></a>
## 5.1.2 Exercise: Build and Run a YOLO Application
Build and run the following sample application to see YOLO in action.  After running the following cells, deepstream will save the result to `/home/dlinano/out.mp4`.  Download `out.mp4` by right clicking it in JupyterLab file browser and select "Download". Then play it on your machine.

In [3]:
%cd /home/dlinano/deepstream_sdk_v4.0.2_jetson/sources/apps/dli_apps/deepstream-test3-mp4_out-yolo/
!make clean
!make

/home/dlinano/deepstream_sdk_v4.0.2_jetson/sources/apps/dli_apps/deepstream-test3-mp4_out-yolo
rm -rf deepstream_test3_app.o deepstream-test3-app
cc -c -o deepstream_test3_app.o -DPLATFORM_TEGRA -I../../../includes `pkg-config --cflags gstreamer-1.0` deepstream_test3_app.c
cc -o deepstream-test3-app deepstream_test3_app.o `pkg-config --libs gstreamer-1.0` -L/opt/nvidia/deepstream/deepstream-4.0/lib/ -lnvdsgst_meta -lnvds_meta -lnvdsgst_helper -lm -lgstrtspserver-1.0 -Wl,-rpath,/opt/nvidia/deepstream/deepstream-4.0/lib/


In [4]:
%cd /home/dlinano/deepstream_sdk_v4.0.2_jetson/sources/apps/dli_apps/deepstream-test3-mp4_out-yolo/
!./deepstream-test3-app \
file:///home/dlinano/deepstream_sdk_v4.0.2_jetson/samples/streams/sample_720p.mp4

/home/dlinano/deepstream_sdk_v4.0.2_jetson/sources/apps/dli_apps/deepstream-test3-mp4_out-yolo

 *** DeepStream: Launched RTSP Streaming at rtsp://localhost:8554/ds-test ***

Now playing: file:///home/dlinano/deepstream_sdk_v4.0.2_jetson/samples/streams/sample_720p.mp4,
Opening in BLOCKING MODE 
Creating LL OSD context new
0:00:01.793875516 11780   0x5580f47930 WARN                 nvinfer gstnvinfer.cpp:515:gst_nvinfer_logger:<primary-nvinference-engine> NvDsInferContext[UID 1]:useEngineFile(): Failed to read from model engine file
0:00:01.793969060 11780   0x5580f47930 INFO                 nvinfer gstnvinfer.cpp:519:gst_nvinfer_logger:<primary-nvinference-engine> NvDsInferContext[UID 1]:initialize(): Trying to create engine from model files
Loading pre-trained weights...
Loading complete!
Total Number of weights read : 8858734
      layer               inp_size            out_size       weightPtr
(1)   conv-bn-leaky     3 x 416 x 416      16 x 416 x 416    496   
(2)   maxpool       

<a name="05_ex_your_video"></a>
## 5.1.3 Exercise: Annotate Your Own Video with YOLO
Since the sample file, `sample_720p.mp4` has mainly cars and people in the scene, you won't see all YOLO has to offer with the 80 objects it can detect.  Try annotating your own video instead of using the provided sample:

1. Import your own video file: drag-and-drop it from your machine to JupyterLab file browser.
2. Run the following cell with the input path of your imported file. 
3. The result will be saved to `/home/dlinano/out.mp4` as before.

In [8]:
%cd /home/dlinano/deepstream_sdk_v4.0.2_jetson/sources/apps/dli_apps/deepstream-test3-mp4_out-yolo/
!./deepstream-test3-app \
file:///home/dlinano/personal/Terminator.mp4

/home/dlinano/deepstream_sdk_v4.0.2_jetson/sources/apps/dli_apps/deepstream-test3-mp4_out-yolo

 *** DeepStream: Launched RTSP Streaming at rtsp://localhost:8554/ds-test ***

Now playing: file:///home/dlinano/personal/Terminator.mp4,
Opening in BLOCKING MODE 
Creating LL OSD context new
Deserialize yoloLayerV3 plugin: yolo_17
Deserialize yoloLayerV3 plugin: yolo_24
Decodebin child added: source
Decodebin child added: decodebin0
Running...
Decodebin child added: qtdemux0
Decodebin child added: multiqueue0
Decodebin child added: aacparse0
Decodebin child added: avdec_aac0
Decodebin child added: h264parse0
Decodebin child added: capsfilter0
Decodebin child added: nvv4l2decoder0
Seting bufapi_version
Opening in BLOCKING MODE 
NvMMLiteOpen : Block : BlockType = 261 
NVMEDIA: Reading vendor.tegra.display-size : status: 6 
NvMMLiteBlockCreate : Block : BlockType = 261 
In cb_newpad
In cb_newpad
Creating LL OSD context new
NvMMLiteOpen : Block : BlockType = 4 
===== NVMEDIA: NVENC =====
NvMMLi

<a name="05_faster_rcnn"></a>
# 5.2 Faster-RCNN

This sample uses TensorRT plugins, performs inference, and implements a fused custom layer for end-to-end inferencing of a Faster R-CNN model. Specifically, this sample demonstrates the implementation of a Faster R-CNN network in TensorRT, performs a quick performance test in TensorRT, implements a fused custom layer, and constructs the basis for further optimization.

Navigate to `/home/dlinano/deepstream_sdk_v4.0.2_jetson/sources/objectDetector_FasterRCNN` using the JupyterLab file browser. Click the README for detailed instructions on DeepStream pre-requisites and usage.
    

<a name="05_ssd"></a>
# 5.3 SSD
    
The SSD, or "Single Shot Detector", network performs the task of object detection and localization in a single forward pass of the network. This network is built using the VGG network as a backbone. Unlike Faster R-CNN, SSD completely eliminates proposal generation and subsequent pixel or feature resampling stages.  It encapsulates all computation in a single network. This makes SSD straightforward to integrate into systems that require a detection component. 

Navigate to `/home/dlinano/deepstream_sdk_v4.0.2_jetson/sources/objectDetector_SSD` using the JupyterLab file browser. Click the README for detailed instructions on DeepStream pre-requisites and usage.

<h2 style="color:green;">Congratulations!</h2>

You ran a DeepStream app using a different network. <br>
Move on to [6.0 Using a Live Stream (Optional)](./06_DSWebcam.ipynb).

<center><img src="images/DLI Header.png" alt="Header" width="400"></center>